In [37]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.layers import Dropout, Conv2D,  MaxPooling2D, Dense, Flatten, Activation, BatchNormalization
from tensorflow.keras import Sequential, optimizers
from tensorflow.keras.regularizers import l2
from tensorflow import config
from skimage.io import imread, imsave
from skimage.transform import resize, rotate
from sklearn.model_selection import train_test_split
import seaborn as sns
import math
from os.path import join
from sklearn.ev

In [2]:
print("GPU is", "available" if config.list_physical_devices('GPU') else "NOT AVAILABLE")

GPU is available


In [69]:
df = pd.read_csv("data/name_images.csv", index_col=0)
df = pd.get_dummies(df, columns=["label"])
df.head()

,fn,label_rotated_left,label_rotated_right,label_upright,label_upside_down
0,down_left86-28987586_1917-03-28_1948.jpg,1,0,0,0
1,left_down60-17051360_1988-08-04_2011.jpg,0,0,0,1
2,63-33314263_1984-06-21_2009.jpg,0,1,0,0
3,right_left14-6093414_1980-11-03_2013.jpg,1,0,0,0
4,54-1421654_1933-08-27_2011.jpg,0,0,1,0


In [70]:
X = df.fn.values
Y = df[['label_rotated_left', 'label_rotated_right', 'label_upright', 'label_upside_down']].values

x, x_test, y, y_test = train_test_split(X, Y, test_size=0.1)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.12)

In [71]:
class BatchGenerator(Sequence) :
  
        def __init__(self, x_set, y_set, batch_size):
            self.x, self.y = x_set, y_set
            self.batch_size = batch_size

        def __len__(self):
            return math.ceil(len(self.x) / self.batch_size)

        def __getitem__(self, idx):
            batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
            batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

            return np.array([imread(join("data","images",file_name))
                   for file_name in batch_x]), np.array(batch_y)

In [72]:
def create_model(learning_rate=0.0001, beta_1=0.80, activation="relu", filters=[32, 32, 64, 64, 512], init_mode='glorot_uniform'):
    model = Sequential()
    model.add(Conv2D(filters[0], (3,3), input_shape=(64,64,3), padding="same"))
    model.add(Activation(activation))
    
    model.add(Conv2D(filters[1], (3, 3), padding="same"))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))

    model.add(Conv2D(filters[2], (3, 3)))
    model.add(Activation(activation))
    model.add(Conv2D(filters[3], (3, 3)))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(filters[4], kernel_regularizer=l2(0.1)))
    model.add(Activation(activation))
    model.add(Dropout(0.25))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    opt = optimizers.Adam(learning_rate=learning_rate, decay=1e-6, beta_1=beta_1)

    model.compile(loss='categorical_crossentropy', optimizer=opt,
                 metrics=['accuracy'])
    
    return model

In [73]:
batch_size = 512
val_batch_generator = BatchGenerator(x_val, y_val, batch_size=batch_size)
train_batch_generator = BatchGenerator(x_train, y_train, batch_size=batch_size)
test_batch_generator = BatchGenerator(x_test, y_test, batch_size=batch_size)

In [74]:
model = create_model(beta_1=0.8, filters=[32, 32, 32, 64, 512])

In [75]:
model.fit(x=train_batch_generator, verbose=1, epochs=10)

  ...
    to  
  ['...']
Train for 303 steps
Epoch 1/10
303/303 [==============================] - 651s 2s/step - loss: 54.5827 - accuracy: 0.7454
Epoch 2/10
303/303 [==============================] - 93s 306ms/step - loss: 23.8853 - accuracy: 0.9012
Epoch 3/10
303/303 [==============================] - 95s 313ms/step - loss: 14.4967 - accuracy: 0.9358
Epoch 4/10
303/303 [==============================] - 105s 347ms/step - loss: 9.2274 - accuracy: 0.9496
Epoch 5/10
303/303 [==============================] - 109s 361ms/step - loss: 5.9605 - accuracy: 0.9594
Epoch 6/10
303/303 [==============================] - 112s 369ms/step - loss: 3.8626 - accuracy: 0.9655
Epoch 7/10
303/303 [==============================] - 108s 355ms/step - loss: 2.4933 - accuracy: 0.9690
Epoch 8/10
303/303 [==============================] - 99s 328ms/step - loss: 1.6026 - accuracy: 0.9716
Epoch 9/10
303/303 [==============================] - 110s 364ms/step - loss: 1.0255 - accuracy: 0.9738
Epoch 10/10
303/303 [=

In [76]:
model.evaluate(x=val_batch_generator)

  ...
    to  
  ['...']
42/42 [==============================] - 90s 2s/step - loss: 0.5177 - accuracy: 0.9774


[0.5177185918603625, 0.977418]

In [77]:
model.evaluate(x=test_batch_generator)

  ...
    to  
  ['...']
39/39 [==============================] - 82s 2s/step - loss: 0.5156 - accuracy: 0.9788


[0.515584218960542, 0.97883326]

In [78]:
y_test_predict = np.argmax(model.predict(x=test_batch_generator), axis=1)

In [79]:
def rotate_image(images, orientations):
    for image_name, orientation in zip(images, orientations):
        image = imread(join("data", "images", image_name))
        if orientation == 0:
            image = rotate(image, -90, preserve_range=True).astype(np.uint8)
        elif orientation == 1:
            image = rotate(image, 90, preserve_range=True).astype(np.uint8)
        elif orientation == 3:
            image = rotate(image, 180, preserve_range=True).astype(np.uint8)
            
            
        imsave((join("data","test", image_name)), image)

In [81]:
import warnings
warnings.filterwarnings("ignore")

# Folder "data/test" must be empty to run the cell bellow!

In [84]:
rotate_image(x_test, y_test_predict)

In [85]:
model.save("model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model\assets
